In [1]:
import numpy as np
from matplotlib import pyplot as plt
from numpy import random as rnd
import warnings,datetime,os,calendar,csv

import tensorflow as tf
import pandas as pd
import seaborn as sns

from keras.models import Model,Sequential
from keras.layers import Dense,LSTM,Conv2D,Dropout,BatchNormalization,Input,Concatenate,Add,Activation,MaxPooling2D,AveragePooling2D
import keras.backend as K

from sklearn import preprocessing as pp

from sklearn.cluster import KMeans,MeanShift
from sklearn.dummy import DummyClassifier,DummyRegressor
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import AdaBoostClassifier,AdaBoostRegressor,RandomForestClassifier,RandomForestRegressor
from sklearn.linear_model import BayesianRidge,Lasso,LinearRegression,SGDClassifier,SGDRegressor
from sklearn.mixture import BayesianGaussianMixture,GaussianMixture
from sklearn.neighbors import KNeighborsClassifier,KNeighborsRegressor,RadiusNeighborsClassifier,RadiusNeighborsRegressor,NearestNeighbors
from sklearn.manifold import Isomap
from sklearn.model_selection import KFold,StratifiedKFold,train_test_split
from sklearn.svm import LinearSVC,LinearSVR
from sklearn.neural_network import BernoulliRBM,MLPClassifier,MLPRegressor
from sklearn.decomposition import FactorAnalysis,KernelPCA,PCA,MiniBatchSparsePCA,FastICA
from sklearn.preprocessing import CategoricalEncoder,KBinsDiscretizer,LabelEncoder,MinMaxScaler,OneHotEncoder,StandardScaler

from xgboost import XGBClassifier,XGBRegressor
import gym

import pickle,h5py,json
import urllib
from bs4 import BeautifulSoup

import pygame
from pygame.locals import *

warnings.filterwarnings('ignore')
sns.set()

C:\Users\Jan\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


pygame 1.9.4
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
class Environment():
    
    def __init__(self,num_obstacles,num_greenbeans=1):
        self.screen = pygame.display.set_mode((1200,600))
        self.clock = pygame.time.Clock()
        
        self.obstacles = []
        for k in range(num_obstacles):
            crect = pygame.rect.Rect(rnd.randint(50,1100,1),rnd.randint(50,500,1),rnd.randint(20,100,1),rnd.randint(20,100,1))
            self.obstacles.append(crect)
        
    def draw_env(self):
        self.screen.fill(0)
        for obstacle in self.obstacles:
            pygame.draw.rect(self.screen,(0,255,0),obstacle)
        
    def draw_player(self,agent):
        brightness = 255
        pygame.draw.rect(self.screen,(0,0,brightness),agent.rect)
        pygame.draw.rect(self.screen,(255,255,255),agent.rect_t)
        pygame.draw.rect(self.screen,(255,255,255),agent.rect_r)
        pygame.draw.rect(self.screen,(255,255,255),agent.rect_l)
        pygame.draw.rect(self.screen,(255,255,255),agent.rect_b)

In [8]:
class Agent():
    
    def __init__(self,pos_init=(200,100),maxstep=5,size=(20,20),spots_offset=5,num_obstacles=5):
        
        self.pos = pos_init
        self.maxstep = maxstep
        self.size = size
        self.spots_offset = spots_offset
        
        self.pain_t = 0
        self.pain_r = 0
        self.pain_l = 0
        self.pain_b = 0
        #self.pain = np.zeros([4,1])
        self.dir = [0,0]
        
        self.rect = pygame.rect.Rect((self.pos[0],self.pos[1],self.size[0],self.size[1]))
        self.rect_t = pygame.rect.Rect((self.pos[0]+self.spots_offset,self.pos[1],self.size[0]-2*self.spots_offset,1))
        self.rect_r = pygame.rect.Rect((self.pos[0]+self.size[0]-1,self.pos[1]+self.spots_offset,1,self.size[1]-2*self.spots_offset))
        self.rect_l = pygame.rect.Rect((self.pos[0],self.pos[1]+self.spots_offset,1,self.size[1]-2*self.spots_offset))
        self.rect_b = pygame.rect.Rect((self.pos[0]+self.spots_offset,self.pos[1]+self.size[1]-1,self.size[0]-2*self.spots_offset,1))
        
    def get_pos(self):
        return (self.rect[0],self.rect[1])
    
    def stop(self):
        self.dir = [0,0]
        
    def softmax(self,x):
        return np.exp(x-np.max(x))/np.sum(np.exp(x-np.max(x)))
        
    def accelerate(self,vvec):
        newdir = [self.dir[0]+vvec[0],self.dir[1]+vvec[1]]
        if newdir[0]<=self.maxstep and newdir[0]>=-self.maxstep:self.dir[0] = newdir[0]
        if newdir[1]<=self.maxstep and newdir[1]>=-self.maxstep:self.dir[1] = newdir[1]
        
    def move(self,env):
        
        img = np.array(pygame.PixelArray(env.screen))
        print(np.where(img!=0))
        screen_rect = env.screen.get_rect()
        
        options = [[0,1],[0,-1],[1,0],[-1,0]]
        self.accelerate(options[rnd.randint(0,4,1)[0]])
        
        if any([self.rect_t.colliderect(o) for o in env.obstacles]) or self.rect_t[1]==0:
            self.pain_t += 1
            self.dir = [0,1]
        else:
            if self.pain_t>0:
                self.pain_t -= 1
        if any([self.rect_r.colliderect(o) for o in env.obstacles]) or self.rect_r[0]+1==1200:
            self.pain_r += 1
            self.dir = [-1,0]
        else:
            if self.pain_r>0:
                self.pain_r -= 1
        if any([self.rect_l.colliderect(o) for o in env.obstacles]) or self.rect_l[0]==0:
            self.pain_l += 1
            self.dir = [1,0]
        else:
            if self.pain_l>0:
                self.pain_l -= 1
        if any([self.rect_b.colliderect(o) for o in env.obstacles]) or self.rect_b[1]+1==600:
            self.pain_b += 1
            self.dir = [0,-1]
        else:
            if self.pain_b>0:
                self.pain_b -= 1
        
        self.rect.move_ip(self.dir[0],0)
        self.rect.move_ip(0,self.dir[1])
        self.rect.clamp_ip(screen_rect)
        
        self.pos = (self.rect[0],self.rect[1]) 
        
        self.rect_t = pygame.rect.Rect((self.pos[0]+self.spots_offset,self.pos[1],self.size[0]-2*self.spots_offset,1))
        self.rect_r = pygame.rect.Rect((self.pos[0]+self.size[0]-1,self.pos[1]+self.spots_offset,1,self.size[1]-2*self.spots_offset))
        self.rect_l = pygame.rect.Rect((self.pos[0],self.pos[1]+self.spots_offset,1,self.size[1]-2*self.spots_offset))
        self.rect_b = pygame.rect.Rect((self.pos[0]+self.spots_offset,self.pos[1]+self.size[1]-1,self.size[0]-2*self.spots_offset,1))

In [9]:
num_obstacles = 15

pygame.init()
env = Environment(num_obstacles)
player = Agent()

try:
    
    running = True
    while running==True:
        
        env.draw_env()
        env.draw_player(player)
        player.move(env)
        pygame.display.flip()
        
        if any([e.type==pygame.KEYDOWN for e in pygame.event.get()]):running=False
        env.clock.tick(50)
        
    pygame.quit()
    
except SystemExit:
    pygame.quit()

(array([ 117,  117,  117, ..., 1026, 1026, 1026], dtype=int64), array([ 72,  73,  74, ..., 505, 506, 507], dtype=int64))
(array([ 117,  117,  117, ..., 1026, 1026, 1026], dtype=int64), array([ 72,  73,  74, ..., 505, 506, 507], dtype=int64))
(array([ 117,  117,  117, ..., 1026, 1026, 1026], dtype=int64), array([ 72,  73,  74, ..., 505, 506, 507], dtype=int64))
(array([ 117,  117,  117, ..., 1026, 1026, 1026], dtype=int64), array([ 72,  73,  74, ..., 505, 506, 507], dtype=int64))
(array([ 117,  117,  117, ..., 1026, 1026, 1026], dtype=int64), array([ 72,  73,  74, ..., 505, 506, 507], dtype=int64))
(array([ 117,  117,  117, ..., 1026, 1026, 1026], dtype=int64), array([ 72,  73,  74, ..., 505, 506, 507], dtype=int64))
(array([ 117,  117,  117, ..., 1026, 1026, 1026], dtype=int64), array([ 72,  73,  74, ..., 505, 506, 507], dtype=int64))
(array([ 117,  117,  117, ..., 1026, 1026, 1026], dtype=int64), array([ 72,  73,  74, ..., 505, 506, 507], dtype=int64))
(array([ 117,  117,  117, ..., 1

(array([ 117,  117,  117, ..., 1026, 1026, 1026], dtype=int64), array([ 72,  73,  74, ..., 505, 506, 507], dtype=int64))
(array([ 117,  117,  117, ..., 1026, 1026, 1026], dtype=int64), array([ 72,  73,  74, ..., 505, 506, 507], dtype=int64))
(array([ 117,  117,  117, ..., 1026, 1026, 1026], dtype=int64), array([ 72,  73,  74, ..., 505, 506, 507], dtype=int64))
(array([ 117,  117,  117, ..., 1026, 1026, 1026], dtype=int64), array([ 72,  73,  74, ..., 505, 506, 507], dtype=int64))
(array([ 117,  117,  117, ..., 1026, 1026, 1026], dtype=int64), array([ 72,  73,  74, ..., 505, 506, 507], dtype=int64))
(array([ 117,  117,  117, ..., 1026, 1026, 1026], dtype=int64), array([ 72,  73,  74, ..., 505, 506, 507], dtype=int64))
(array([ 117,  117,  117, ..., 1026, 1026, 1026], dtype=int64), array([ 72,  73,  74, ..., 505, 506, 507], dtype=int64))
(array([ 117,  117,  117, ..., 1026, 1026, 1026], dtype=int64), array([ 72,  73,  74, ..., 505, 506, 507], dtype=int64))
(array([ 117,  117,  117, ..., 1

(array([ 117,  117,  117, ..., 1026, 1026, 1026], dtype=int64), array([ 72,  73,  74, ..., 505, 506, 507], dtype=int64))
(array([ 117,  117,  117, ..., 1026, 1026, 1026], dtype=int64), array([ 72,  73,  74, ..., 505, 506, 507], dtype=int64))
(array([ 117,  117,  117, ..., 1026, 1026, 1026], dtype=int64), array([ 72,  73,  74, ..., 505, 506, 507], dtype=int64))
(array([ 117,  117,  117, ..., 1026, 1026, 1026], dtype=int64), array([ 72,  73,  74, ..., 505, 506, 507], dtype=int64))
(array([ 117,  117,  117, ..., 1026, 1026, 1026], dtype=int64), array([ 72,  73,  74, ..., 505, 506, 507], dtype=int64))
(array([ 117,  117,  117, ..., 1026, 1026, 1026], dtype=int64), array([ 72,  73,  74, ..., 505, 506, 507], dtype=int64))
(array([ 117,  117,  117, ..., 1026, 1026, 1026], dtype=int64), array([ 72,  73,  74, ..., 505, 506, 507], dtype=int64))
(array([ 117,  117,  117, ..., 1026, 1026, 1026], dtype=int64), array([ 72,  73,  74, ..., 505, 506, 507], dtype=int64))
(array([ 117,  117,  117, ..., 1

(array([ 117,  117,  117, ..., 1026, 1026, 1026], dtype=int64), array([ 72,  73,  74, ..., 505, 506, 507], dtype=int64))
(array([ 117,  117,  117, ..., 1026, 1026, 1026], dtype=int64), array([ 72,  73,  74, ..., 505, 506, 507], dtype=int64))
(array([ 117,  117,  117, ..., 1026, 1026, 1026], dtype=int64), array([ 72,  73,  74, ..., 505, 506, 507], dtype=int64))
(array([ 117,  117,  117, ..., 1026, 1026, 1026], dtype=int64), array([ 72,  73,  74, ..., 505, 506, 507], dtype=int64))
(array([ 117,  117,  117, ..., 1026, 1026, 1026], dtype=int64), array([ 72,  73,  74, ..., 505, 506, 507], dtype=int64))
(array([ 117,  117,  117, ..., 1026, 1026, 1026], dtype=int64), array([ 72,  73,  74, ..., 505, 506, 507], dtype=int64))
(array([ 117,  117,  117, ..., 1026, 1026, 1026], dtype=int64), array([ 72,  73,  74, ..., 505, 506, 507], dtype=int64))
(array([ 117,  117,  117, ..., 1026, 1026, 1026], dtype=int64), array([ 72,  73,  74, ..., 505, 506, 507], dtype=int64))
(array([ 117,  117,  117, ..., 1

(array([  99,   99,   99, ..., 1026, 1026, 1026], dtype=int64), array([ 19,  20,  21, ..., 505, 506, 507], dtype=int64))
(array([  94,   94,   94, ..., 1026, 1026, 1026], dtype=int64), array([ 16,  17,  18, ..., 505, 506, 507], dtype=int64))
(array([  89,   89,   89, ..., 1026, 1026, 1026], dtype=int64), array([ 12,  13,  14, ..., 505, 506, 507], dtype=int64))
(array([  84,   84,   84, ..., 1026, 1026, 1026], dtype=int64), array([  7,   8,   9, ..., 505, 506, 507], dtype=int64))
(array([  79,   79,   79, ..., 1026, 1026, 1026], dtype=int64), array([  2,   3,   4, ..., 505, 506, 507], dtype=int64))
(array([  74,   74,   74, ..., 1026, 1026, 1026], dtype=int64), array([  0,   1,   2, ..., 505, 506, 507], dtype=int64))
(array([  74,   74,   74, ..., 1026, 1026, 1026], dtype=int64), array([  1,   2,   3, ..., 505, 506, 507], dtype=int64))
(array([  73,   73,   73, ..., 1026, 1026, 1026], dtype=int64), array([  2,   3,   4, ..., 505, 506, 507], dtype=int64))
(array([  72,   72,   72, ..., 1

...

(array([ 117,  117,  117, ..., 1026, 1026, 1026], dtype=int64), array([ 72,  73,  74, ..., 505, 506, 507], dtype=int64))
(array([ 117,  117,  117, ..., 1026, 1026, 1026], dtype=int64), array([ 72,  73,  74, ..., 505, 506, 507], dtype=int64))
(array([ 117,  117,  117, ..., 1026, 1026, 1026], dtype=int64), array([ 72,  73,  74, ..., 505, 506, 507], dtype=int64))
(array([ 117,  117,  117, ..., 1026, 1026, 1026], dtype=int64), array([ 72,  73,  74, ..., 505, 506, 507], dtype=int64))
(array([ 117,  117,  117, ..., 1026, 1026, 1026], dtype=int64), array([ 72,  73,  74, ..., 505, 506, 507], dtype=int64))
(array([ 117,  117,  117, ..., 1026, 1026, 1026], dtype=int64), array([ 72,  73,  74, ..., 505, 506, 507], dtype=int64))
(array([ 117,  117,  117, ..., 1026, 1026, 1026], dtype=int64), array([ 72,  73,  74, ..., 505, 506, 507], dtype=int64))
(array([ 117,  117,  117, ..., 1026, 1026, 1026], dtype=int64), array([ 72,  73,  74, ..., 505, 506, 507], dtype=int64))
(array([ 117,  117,  117, ..., 1

(array([ 117,  117,  117, ..., 1026, 1026, 1026], dtype=int64), array([ 72,  73,  74, ..., 505, 506, 507], dtype=int64))
(array([ 117,  117,  117, ..., 1026, 1026, 1026], dtype=int64), array([ 72,  73,  74, ..., 505, 506, 507], dtype=int64))
(array([ 117,  117,  117, ..., 1026, 1026, 1026], dtype=int64), array([ 72,  73,  74, ..., 505, 506, 507], dtype=int64))
(array([ 117,  117,  117, ..., 1026, 1026, 1026], dtype=int64), array([ 72,  73,  74, ..., 505, 506, 507], dtype=int64))
(array([ 117,  117,  117, ..., 1026, 1026, 1026], dtype=int64), array([ 72,  73,  74, ..., 505, 506, 507], dtype=int64))
(array([ 117,  117,  117, ..., 1026, 1026, 1026], dtype=int64), array([ 72,  73,  74, ..., 505, 506, 507], dtype=int64))
(array([ 117,  117,  117, ..., 1026, 1026, 1026], dtype=int64), array([ 72,  73,  74, ..., 505, 506, 507], dtype=int64))
(array([ 117,  117,  117, ..., 1026, 1026, 1026], dtype=int64), array([ 72,  73,  74, ..., 505, 506, 507], dtype=int64))
(array([ 117,  117,  117, ..., 1

(array([ 117,  117,  117, ..., 1026, 1026, 1026], dtype=int64), array([ 72,  73,  74, ..., 505, 506, 507], dtype=int64))
(array([ 117,  117,  117, ..., 1026, 1026, 1026], dtype=int64), array([ 72,  73,  74, ..., 505, 506, 507], dtype=int64))
(array([ 117,  117,  117, ..., 1026, 1026, 1026], dtype=int64), array([ 72,  73,  74, ..., 505, 506, 507], dtype=int64))
(array([ 117,  117,  117, ..., 1026, 1026, 1026], dtype=int64), array([ 72,  73,  74, ..., 505, 506, 507], dtype=int64))
(array([ 117,  117,  117, ..., 1026, 1026, 1026], dtype=int64), array([ 72,  73,  74, ..., 505, 506, 507], dtype=int64))
(array([ 117,  117,  117, ..., 1026, 1026, 1026], dtype=int64), array([ 72,  73,  74, ..., 505, 506, 507], dtype=int64))
(array([ 117,  117,  117, ..., 1026, 1026, 1026], dtype=int64), array([ 72,  73,  74, ..., 505, 506, 507], dtype=int64))
(array([ 117,  117,  117, ..., 1026, 1026, 1026], dtype=int64), array([ 72,  73,  74, ..., 505, 506, 507], dtype=int64))
(array([ 117,  117,  117, ..., 1

(array([ 117,  117,  117, ..., 1026, 1026, 1026], dtype=int64), array([ 72,  73,  74, ..., 505, 506, 507], dtype=int64))
(array([ 117,  117,  117, ..., 1026, 1026, 1026], dtype=int64), array([ 72,  73,  74, ..., 505, 506, 507], dtype=int64))
(array([ 117,  117,  117, ..., 1026, 1026, 1026], dtype=int64), array([ 72,  73,  74, ..., 505, 506, 507], dtype=int64))
(array([ 117,  117,  117, ..., 1026, 1026, 1026], dtype=int64), array([ 72,  73,  74, ..., 505, 506, 507], dtype=int64))
(array([ 117,  117,  117, ..., 1026, 1026, 1026], dtype=int64), array([ 72,  73,  74, ..., 505, 506, 507], dtype=int64))
(array([ 117,  117,  117, ..., 1026, 1026, 1026], dtype=int64), array([ 72,  73,  74, ..., 505, 506, 507], dtype=int64))
(array([ 117,  117,  117, ..., 1026, 1026, 1026], dtype=int64), array([ 72,  73,  74, ..., 505, 506, 507], dtype=int64))
(array([ 117,  117,  117, ..., 1026, 1026, 1026], dtype=int64), array([ 72,  73,  74, ..., 505, 506, 507], dtype=int64))
(array([ 117,  117,  117, ..., 1

(array([ 117,  117,  117, ..., 1026, 1026, 1026], dtype=int64), array([ 72,  73,  74, ..., 505, 506, 507], dtype=int64))
(array([ 117,  117,  117, ..., 1026, 1026, 1026], dtype=int64), array([ 72,  73,  74, ..., 505, 506, 507], dtype=int64))
(array([ 117,  117,  117, ..., 1026, 1026, 1026], dtype=int64), array([ 72,  73,  74, ..., 505, 506, 507], dtype=int64))
(array([ 117,  117,  117, ..., 1026, 1026, 1026], dtype=int64), array([ 72,  73,  74, ..., 505, 506, 507], dtype=int64))
(array([ 117,  117,  117, ..., 1026, 1026, 1026], dtype=int64), array([ 72,  73,  74, ..., 505, 506, 507], dtype=int64))
(array([ 117,  117,  117, ..., 1026, 1026, 1026], dtype=int64), array([ 72,  73,  74, ..., 505, 506, 507], dtype=int64))
(array([ 117,  117,  117, ..., 1026, 1026, 1026], dtype=int64), array([ 72,  73,  74, ..., 505, 506, 507], dtype=int64))
(array([ 117,  117,  117, ..., 1026, 1026, 1026], dtype=int64), array([ 72,  73,  74, ..., 505, 506, 507], dtype=int64))
(array([ 117,  117,  117, ..., 1